## Table of Contents
1. [Instructions](#Instructions)
1. [Import Statements](#Import-Statements)
1. [Functions](#Functions)

# Instructions:
1. PFRTP-Garrett must be run with the same conditions first, as the species used in the path diagram for CFS are the species in the path diagram made by Cantera.
1. Set injected_species to the species used in the simulation
1. Set netj_file to the .netj file that was used for the simulation
1. Run all

# Import Statements

In [1]:
import sys
import cantera as ct
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os as os
import ipywidgets as widgets
from ipywidgets import widgets, interact
import matplotlib.pyplot as plt
import matplotlib.image as mpimage
import subprocess
import re
import graphviz

In [6]:
mechanism = "NCSU_PFASmech1.0.yaml"
plt.rcParams["figure.dpi"] = 200
gas = ct.Solution(mechanism)

thresh_min = 0.0
thresh_max = 1.0
thresh_step = 1e-2
cutoff = 1e-20
tolerance = 0.0001
injected_species = 'CF4'

########## Path diagram ##########
path_element = 'F' # case sensitive

########## File paths ##########
# Set the path to the netj file to be analyzed
netj_file = '14.45.16_45kW_CF4_port4_16stream.netj'
run_folder = netj_file.replace('.netj','')
main_dir = os.path.join(os.getenv('USERPROFILE'), 'Desktop', 'PFAS_Modeling', 'cfs_runs')
output_folder = os.path.join(main_dir, run_folder, 'reaction_paths')
avg_file = os.path.join(main_dir, run_folder, 'excel\\avg_data.csv')
vtk_path  = os.path.join(main_dir, run_folder, 'streamline_pp.vtk')
pvpython_path = "C:\\Program Files\\ParaView 5.11.2\\bin\\pvpython.exe"
vtk_path = vtk_path.replace('\\','/')
destruct_path = os.path.join(main_dir, run_folder, 'excel\\destruct.csv')
destruct_path = destruct_path.replace('\\','/')
destruct_file = os.path.join(main_dir, run_folder, 'excel\\destruct.csv')
destruct_file = destruct_file.replace('\\','/')
script_content = "from paraview.simple import *\nreader = OpenDataFile('" + vtk_path + "')\nSaveData('"+ destruct_path + "', proxy=reader, Precision=15)"
script_dir = os.path.join(main_dir, run_folder, 'excel\\convert_script.py')


pattern = r"(\d{2}\.\d{2,3})\.\d+(_[^_]+_[^_]+_[^_]+)_\d+stream\.netj$"
replacement = r"\1\2/dotfile.dot"
new_base_name = re.sub(pattern, replacement, netj_file)

# Join the modified filename with the original directory
dot_file = os.path.join(os.getenv('USERPROFILE'),'Desktop','PFAS_Modeling','pseudo_PFR','Output', new_base_name).replace('\\','/')


########## Species list ##########
def get_species_names(dot_file):
    #read the dot file
    dot_file = graphviz.Source.from_file(dot_file)
    #make regex to find the labels, returning only the species names. The regex looks for the letters and numbers between label=" and ", while ignoring values that don't have a letter in them
    regex = r'(?<=label=")[A-Za-z0-9]+(?=")'

    #use regex to get labels from the source
    labels = dot_file.source
    #get the labels from the source using regex
    return(re.findall(regex, labels))

spec_list = get_species_names(dot_file)
#if entry in species list is only one character, add to element list
elem_list = [spec_list[i] for i in range(len(spec_list)) if len(spec_list[i]) == 1] #TODO get elements from within compounds instead of just plain elements


In [3]:
#Converting the vtk file to a usable data frame

#run pvpython from command line with subprocess
def run_pvpython():
    subprocess.run([pvpython_path, script_dir], shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text = 'TRUE')

if not os.path.exists(destruct_file): #check if destruct.csv exists
    print('No destruct.csv, creating now') #if not, create it
    if not os.path.exists(script_dir): #check if convert_script.py exists
        print('No convert_script.py, creating now') #if not, create it
        with open(script_dir, "w") as file: #write script to convert vtk to csv
                file.write(script_content)
                run_pvpython() #convert vtk to csv
    else: #if convert_script.py already exists, run pvpython
        print('convert_script.py already exists, skipping creation')
        run_pvpython() #convert vtk to csv
else: #if destruct.csv already exists, skip creation
    print('Destruct.csv already exists, skipping creation')

data = pd.read_csv(destruct_file, sep = ',', header = 0, index_col = False)
data.rename(columns = {'OHr':'OH'}, inplace = True)
traj_numbers = data['traj_number'].unique()



Destruct.csv already exists, skipping creation


# Functions

In [7]:
def plot_reaction_path_diagrams(traj_num, res_time, threshold, details, species):
    """Plot reaction path diagram for a given trajectory number and residence time"""
    #select only rows from avg_data with traj_num = traj_num
    global plot_data
    plot_data = data[data['traj_number'] == traj_num] #select only rows from data with traj_num = traj_num
    plot_data = plot_data[plot_data[injected_species] > cutoff]
    plot_data = calc_res_time(plot_data, cutoff)
    global initial_row
    initial_row = plot_data[
        (plot_data['residence_time'] >= res_time - tolerance) &
        (plot_data['residence_time'] <= res_time + tolerance)
     ].iloc[0].to_frame().T #select row with residence time = res_time
    global species_dict
    species_dict = initial_row.drop(['time', 'TEMPERATURE:','PRESSURE:','MEAN_MW:','walls', 'traj_number','residence_time','Points:0', 'Points:1', 'Points:2'],axis= 'columns').iloc[0].to_dict()

    P = ct.one_atm
    T = initial_row['TEMPERATURE:']
    X = species_dict
    gas.TP = float(T.iloc[0]), P
    gas.X = X

    diagram = ct.ReactionPathDiagram(gas, species)
    diagram.threshold = threshold
    diagram.dot_options='node[shape="box"]'
    diagram.show_details = details
    diagram.display_only(-1)
    title = (
        (
            f'Path diagram for {species} at residence time of {str(format_float(res_time))}'
            + ' seconds \r scaled by maximum flux (kmol m-^3 s^-1) \r'
        )
        + ' with displayed threshold of '
    ) + str(threshold)
    diagram.title = title
    diagram.label_threshold = 0.0001
    log_file = f"{output_folder}/reaction_paths.log"
    with open(log_file, 'w') as f:
        sys.stdout = f
        diagram.build(verbose=True)
        sys.stdout = sys.__stdout__
    dot_file = f"{output_folder}/dotfile.dot"
    png_file = f"{output_folder}/reaction_paths.png"
    diagram.write_dot(dot_file)
    os.system(f'dot {dot_file} -Tpng -o {png_file} -Gdpi=300')
    return png_file

def calc_res_time(df, cutoff):
    '''Calculate the residence time for each row in the DataFrame'''
    # find the time of injection for the trajectory
    injection_time = df.loc[df[injected_species] > cutoff, 'time'].min()

    # create a copy of the original DataFrame
    new_df = df.loc[df[injected_species] > cutoff].copy()

    # calculate the residence time for each row
    new_df['residence_time'] = new_df['time'] - injection_time

    return new_df

def validate_and_reset_indices(traj_data):
    ''' Validate that the index of the DataFrame is equal to the range of the length of the DataFrame and reset the index if necessary '''
    if not np.array_equal(traj_data.index, np.arange(len(traj_data))):
        traj_data = traj_data.reset_index(drop=True)
        return traj_data
    
def format_float(num):
    return np.format_float_positional(num, precision=6, fractional=False)

def traj_data_and_res_time(traj_num, res_time):
    ''' Select the trajectory data for the given trajectory number and residence time '''
    traj_data = data[data['traj_number'] == traj_num] #select only rows from avg_data with traj_num = traj_num
    traj_data = calc_res_time(traj_data, cutoff) #calculate residence time, add to traj_data
    traj_data = traj_data.reset_index(drop=True) #reset index 
    if res_time < len(traj_data['residence_time']): 
        res_time_row = traj_data.iloc[res_time] #select row with residence time = res_time
        actual_res_time = res_time_row.loc['residence_time'] #get actual residence time from row
    else:
        actual_res_time = 0
    res_time_str = format_float(actual_res_time)
    res_time_slider.description = f"Residence Time: {res_time_str}"
    return traj_data, actual_res_time

def update_time_description(change):
    ''' Update the residence time description when the slider value changes '''
    traj_num = traj_dropdown.value
    traj_data_and_res_time(traj_num, change.new)


#making interactive options
traj_dropdown = widgets.Dropdown(options = traj_numbers, value = traj_numbers[0], description = 'Trajectory:')
resmax = len(data[data['traj_number'] == traj_dropdown.value])
res_time_slider = widgets.IntSlider(min=0, max=resmax-1, step=1, value=1, 
                                    description='Residence Time: 1.0', 
                                    style = {'description_width': 'initial'}, 
                                    layout = widgets.Layout(width = '50%'), 
                                    continuous_update = False, 
                                    readout = True)
res_time_slider.observe(update_time_description, names='value')
threshold_slider = widgets.FloatSlider(min = thresh_min, max = thresh_max, step = thresh_step, value = 0.01, description = 'Threshold:', readout_format = '.4f' )
details_toggle = widgets.ToggleButton(value = False, description = 'Details:')
species_dropdown = widgets.Dropdown(options = elem_list, value = 'F', description = 'Species:')

@interact(traj_num = traj_dropdown, res_time = res_time_slider, threshold = threshold_slider, details = details_toggle, species = species_dropdown)

def update_plot(traj_num, res_time, threshold, details, species):
    ''' Update the plot when the trajectory number, residence time, threshold, details, or species changes '''
    plt.close('all')
    global traj_data
    traj_data, actual_res_time = traj_data_and_res_time(traj_num, res_time)
    resmax = len(traj_data['residence_time'])
    res_time_slider.max = resmax-1
    
    img = mpimage.imread(plot_reaction_path_diagrams(traj_num, actual_res_time, threshold, details, species))
    
    plt.imshow(img)
    plt.axis('off')


interactive(children=(Dropdown(description='Trajectory:', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1…